In [1]:
#!pip install "transformers" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.23.0" "bitsandbytes" "trl==0.4.7" "safetensors>=0.3.1" --upgrade
! pip install accelerate
! pip install bitsandbytes
! pip install transformers
! pip install datasets
! pip install trl
! pip install safetensors
! pip install peft
! pip install evaluate
! pip install rouge-score
#! pip install torch --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Device Found : {device}")


Device Found : cuda


In [3]:
!nvidia-smi

Tue Oct 24 10:35:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("ShashiVish/cover-letter-dataset", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])



Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

dataset size: 813
{'Job Title': 'Data Scientist', 'Preferred Qualifications': ' Experience with distributed, columnar, and/or analyticoriented databases or distributed data processing frameworks\n Experience with B2B enterprisegrade support\n Practical experience of information technologies (e.g., HTTP/HTML, Operating Systems, Data Warehousing, or Networking)\n Troubleshooting experience with cloud technologies (i.e. Google Cloud Platform)\n Experience with open source distributed storage and processing utilities in the Apache Hadoop family and/or workflow orchestration products', 'Hiring Company': 'XYZ Tech Solutions', 'Applicant Name': 'John Smith', 'Past Working Experience': 'Data Analyst at ABC Company', 'Current Working Experience': 'Data Scientist at XYZ Tech Solutions', 'Skillsets': ' hard skills: data analytics, etl development, apache spark, troubleshooting cloud technologies\n soft skills: customer empathy, effective communication, problemsolving', 'Qualifications': "Bachelor

In [6]:
def format_instruction(item):
	return f"""### Instruction:
You are a smart cover letter generator. Use following Input to generate Cover letter.

### Input:
Role: {item['Job Title']}, Preferred Qualifications: {item['Preferred Qualifications']}, \
                        Hiring Company: {item['Hiring Company']}, User Name: {item['Applicant Name']}, \
                        Past Working Experience: {item['Past Working Experience']}, \
                        Current Working Experience: {item['Current Working Experience']}, \
                        Skillsets: {item['Skillsets']}, Qualifications: {item['Qualifications']}

### Cover Letter :
{item['Cover Letter']}

"""


In [7]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))


### Instruction:
You are a smart cover letter generator. Use following Input to generate Cover letter.

### Input:
Role: Machine Learning Engineer, Preferred Qualifications: PhD in Computer Science,                         Hiring Company: Google, User Name: Oliver Taylor,                         Past Working Experience: Software Engineer at Facebook for 5 years,                         Current Working Experience: Machine Learning Engineer at Microsoft for 2 years,                         Skillsets: Python, Java, Machine Learning, Artificial Intelligence, Qualifications: PhD in Computer Science

### Cover Letter :
I am writing to express my interest in the Machine Learning Engineer position at Google. I have a PhD in Computer Science and have been working as a Machine Learning Engineer for the past two years at Microsoft. Prior to that, I was a Software Engineer at Facebook for five years. My skillset includes Python, Java, Machine Learning, and Artificial Intelligence. I am confident t

In [14]:
import torch

model_name = "llama-7-int4-r512-cover-letter"
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################


# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False # if A100

max_steps=120


compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf"  # non-gated
#model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16 , load_in_8bit = False
# )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto"

)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=512,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

# r 64 - trainable params: 33554432 || all params: 3533967360 || trainable %: 0.9494833591219133
# r8  - trainable params: 4194304 || all params: 3504607232 || trainable %: 0.11967971650867153
# r512 - trainable params: 268435456 || all params: 3768848384 || trainable%: 7.122479565365291

trainable params: 268435456 || all params: 3768848384 || trainable%: 7.122479565365291


In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=bf16,
    fp16=fp16,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    max_steps=max_steps,
    #disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [16]:
from trl import SFTTrainer

max_seq_length = 512 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:234: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [17]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,1.592400
10,1.434800
15,1.238700
20,1.053600
25,0.963800
30,0.914600
35,0.878100
40,0.903200


Step,Training Loss
5,1.592400
10,1.434800
15,1.238700
20,1.053600
25,0.963800
30,0.914600
35,0.878100
40,0.903200
45,0.813700
50,0.815500


### Publish Model on Huggingface

In [18]:
# Publish to Huggingface
from huggingface_hub import login
login()

In [19]:
# save model
trainer.save_model()

In [21]:
from peft import AutoPeftModelForCausalLM


# print(model_path)
model = AutoPeftModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
)

# # Merge LoRA and base model
merged_model = model.merge_and_unload()

# # Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:508: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.json')

In [22]:
# push merged model to the hub
merged_model.push_to_hub("ShashiVish/llama-7b-merged-int4-r512-cover-letter")
tokenizer.push_to_hub("ShashiVish/llama-7b-merged-int4-r512-cover-letter")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:508: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ShashiVish/llama-7b-merged-int4-r512-cover-letter/commit/719ddc88fbc37f3ef826b835742822f65badeacf', commit_message='Upload tokenizer', commit_description='', oid='719ddc88fbc37f3ef826b835742822f65badeacf', pr_url=None, pr_revision=None, pr_num=None)

In [36]:
from huggingface_hub import HfApi

api = HfApi()
model_path = "llama-7-int4-cover-letter"
model_repo_name = "ShashiVish/llama-7-int4-r512-cover-letter"  # Format of Input  <Profile Name > / <Model Repo Name>

#Create Repo in Hugging Face
api.create_repo(repo_id=model_repo_name)

#Upload Model folder from Local to HuggingFace
api.upload_folder(
    folder_path=model_path,
    repo_id=model_repo_name
)

# Publish Model Tokenizer on Hugging Face
tokenizer.push_to_hub(model_repo_name)

adapter_model.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

events.out.tfevents.1698121638.203f97f492a6.410.0:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShashiVish/llama-7-int4-r512-cover-letter/commit/2ea2d0f9f5c1dfea84e89c70321a4fde74603948', commit_message='Upload tokenizer', commit_description='', oid='2ea2d0f9f5c1dfea84e89c70321a4fde74603948', pr_url=None, pr_revision=None, pr_num=None)

### Download and evaluate Model

In [4]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

#model_name = "ShashiVish/llama-7-int4-cover-letter" #Fine Tune Model
model_name = "ShashiVish/llama-7-int4-r512-cover-letter"


# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Shift model to Device
model = model.to(device)

OSError: ignored

In [1]:
#Evalucate with merged complete model
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "ShashiVish/llama-7b-merged-int4-r512-cover-letter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


# Shift model to Device
# model = model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from random import randrange
import pandas as pd
# # Load dataset from the hub and get a sample

dataset = load_dataset("ShashiVish/cover-letter-dataset", split="test")
item = dataset[randrange(len(dataset))]

predictions = []
actuals = []

total_to_process =  len(dataset)


df = pd.read_excel("data_cleaned_test-data_10rows_format2.xlsx")

# Iterate through the dataset
for index, item in df.iterrows():
#for item in dataset:
  prompt = f"""### Instruction:
  You are a smart cover letter generator. Use following Input to generate Cover letter.

  ### Input:
  Role: {item['Job Title']}, Preferred Qualifications: {item['Preferred Qualifications']}, \
                          Hiring Company: {item['Hiring Company']}, User Name: {item['Applicant Name']}, \
                          Past Working Experience: {item['Past Working Experience']}, \
                          Current Working Experience: {item['Current Working Experience']}, \
                          Skillsets: {item['Skillsets']}, Qualifications: {item['Qualifications']}

  ### Cover Letter:
  """

  input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids #.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.9,temperature=0.9)
  #model_response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
  model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][len(prompt):]

  ground_truth = item['Cover Letter']

  predictions.append(model_response)
  actuals.append(ground_truth)
  total_to_process -=1
  print(f"Pending Process  : {total_to_process} ")




Pending Process  : 348 
Pending Process  : 347 
Pending Process  : 346 
Pending Process  : 345 
Pending Process  : 344 
Pending Process  : 343 
Pending Process  : 342 
Pending Process  : 341 
Pending Process  : 340 
Pending Process  : 339 


In [6]:
predictions[2]

'\nDear Hiring Manager,\n\nI am writing to express my interest in the Data Scientist position at XYZ Corporation. With a strong background in data analysis and machine learning, I believe I am well-suited for this role.\n\nIn my current position as a Data Scientist at DEF Company, I have successfully developed and implemented machine learning algorithms, providing valuable insights to our team. My experience with Python, R, Scala, and other programming languages has allowed me to effectively analyze data and communicate findings to stakeholders.\n\nI have excellent written and verbal communication skills, which I use to facilitate discussions with team members and ensure that we are aligned in our goals. Additionally, my interpersonal skills have allowed me to effectively negotiate and manage relationships with vendors and suppliers.\n\nI am confident that my skills and qualifications make me a strong candidate for the Data Scientist position at XYZ Corporation. I am eager to bring my 

In [10]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({'Predictions': predictions, 'Actuals': actuals})

# Save the DataFrame to a CSV file
df.to_csv('llama2_predictions_vs_actuals_512.csv', index=False)


In [11]:
df = pd.read_csv('llama2_predictions_vs_actuals_512.csv')

predictions = df['Predictions']
actuals = df['Actuals']

### Generate Score for Model

In [7]:
# Generate Score.
import evaluate

def get_accuracy_score(predictions, actuals):
  # Calculate BLEU Score

  bleu = evaluate.load('bleu')
  rouge = evaluate.load('rouge')

  score = {}

  results = rouge.compute(predictions=predictions, references=actuals)
  bleu_results =  bleu.compute(predictions=predictions, references=actuals)

  score['bleu'] = bleu_results['bleu']
  score['rouge1'] = results['rouge1']
  score['rouge2'] = results['rouge2']
  score['rougeL'] = results['rougeL']
  score['rougeLsum'] = results['rougeLsum']

  return score



In [12]:
after_fine_tuning =  get_accuracy_score(predictions, actuals)

In [9]:
after_fine_tuning

{'bleu': 0.2765068433715912,
 'rouge1': 0.4863361188303442,
 'rouge2': 0.2819820846207991,
 'rougeL': 0.347834336244107,
 'rougeLsum': 0.41227792560295506}

In [13]:
after_fine_tuning

{'bleu': 0.2765068433715912,
 'rouge1': 0.4863361188303442,
 'rouge2': 0.2819820846207991,
 'rougeL': 0.347834336244107,
 'rougeLsum': 0.41227792560295506}

In [14]:
 pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-6e1wz7n1
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-6e1wz7n1
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=4a2cf87ae9190f9488ce51da6bbee6ab931b355a83c81da99b32f30a00d36439
  Stored in directory: /tmp/pip-ephem-wheel-cache-dkjp3096/wheels/57/de/9f/7aa9ae75f4e4733cc4b66b5c3a65a05de62a28e8111649fee5
Successfully built pycocoevalcap


In [20]:
import pycocoevalcap
from pycocoevalcap.cider.cider import Cider

def calculate_cider_score(generated_summary, human_written_summary):
    """Calculates the CIDEr score between two text summaries.

    Args:
        generated_summary: The generated summary.
        human_written_summary: The human-written summary.

    Returns:
        The CIDEr score.
    """
    actual = {}
    pred = {}
    pred[1] = generated_summary
    actual[1] = human_written_summary
    cider_scorer = Cider()
    cider_score = cider_scorer.compute_score(pred, actual)

    print(cider_score)

    return cider_score

def calculate_meteor_score(generated_summary, human_written_summary):
    """Calculates the METEOR score between two text summaries.

    Args:
        generated_summary: The generated summary.
        human_written_summary: The human-written summary.

    Returns:
        The METEOR score.
    """

    meteor_scorer = pycocoevalcap.Meteor()
    meteor_score = meteor_scorer.compute_score([generated_summary], [human_written_summary])[0]

    return meteor_score

# Example usage:

generated_summary = "A black and white dog running through a field of grass."
human_written_summary = "A black and white Border Collie is running through a field of green grass."

cider_score = calculate_cider_score(generated_summary, human_written_summary)
meteor_score = calculate_meteor_score(generated_summary, human_written_summary)

print("CIDEr score:", cider_score)
print("METEOR score:", meteor_score)


AssertionError: ignored